# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686592


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:11,  2.65s/it]

Rendering models:  13%|█▎        | 4/30 [00:09<00:56,  2.17s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:41,  1.67s/it]

Rendering models:  20%|██        | 6/30 [00:10<00:32,  1.36s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:23<00:01,  1.58it/s]

Rendering models:  93%|█████████▎| 28/30 [00:23<00:01,  1.59it/s]

Rendering models:  97%|█████████▋| 29/30 [00:24<00:00,  1.38it/s]

Rendering models: 100%|██████████| 30/30 [00:25<00:00,  1.46it/s]

not-logged-in-20a275b53abec8db9f41    3.908695
CThomas                               0.000065
EvalynGJ                              0.000036
Dynamatt                              0.000038
Staceyann116                          0.000590
djswanso                              0.000034
not-logged-in-c3e2bb63f9ec3bdeea91    0.000285
bldelacr                              0.000060
tingard                               0.000091
not-logged-in-2bc079ac48efb4acdd24    0.001333
not-logged-in-f2eb212bec3c0ec6797b    0.036088
KjellN                                0.000033
not-logged-in-15c63a3909f8669e205d    0.000637
Lavadude                              0.001788
Windstar187                           0.003122
w7250369                              0.000048
not-logged-in-5300255fb93f0a882568    0.000175
Knight6409                            0.000666
not-logged-in-fc3fc98dca6b1a783771    0.002392
syt18                                 0.000075
Mr_DMan                               0.000204
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())